In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [6]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')

print("Train data head:")
print(df_train.head())

print("\nTrain data info:")
print(df_train.info())

df_test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')

print("\nTest data head:")
print(df_test.head())

print("\nTest data info:")
print(df_test.info())

df_sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

print("\nSample Submission data head:")
print(df_sample_submission.head())

print("\nSample Submission data info:")
print(df_sample_submission.info())

Train data head:
   id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0  cellular   25   aug       117         3     -1         0  unknown  0  
1   unknown   18   jun       185         1     -1         0  unknown  0  
2   unknown   14   may       111         2     -1         0  unknown  0  
3   unknown   28   may        10         2     -1         0  unknown  0  
4  cellular    3   feb       902         1     -1         0  unknown  1  

Train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 e

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [8]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
df_sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

In [9]:
X = df_train.drop(columns=['id', 'y'])
y = df_train['y']

In [10]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [11]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [13]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression(solver='liblinear', random_state=42, n_jobs=-1))])

In [14]:
print("Training the Logistic Regression model...")
model.fit(X, y)
print("Model training complete.")

Training the Logistic Regression model...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


Model training complete.


In [15]:
X_test = df_test.drop(columns=['id'])
test_predictions_proba = model.predict_proba(X_test)[:, 1]

In [16]:
df_submission = pd.DataFrame({'id': df_test['id'], 'y': test_predictions_proba})

In [17]:
submission_file_path = 'submission.csv'
df_submission.to_csv(submission_file_path, index=False)

In [18]:
print(f"\nSubmission file created: {submission_file_path}")
print("First 5 rows of the submission file:")
print(df_submission.head())


Submission file created: submission.csv
First 5 rows of the submission file:
       id         y
0  750000  0.007759
1  750001  0.324820
2  750002  0.007733
3  750003  0.002026
4  750004  0.029475


In [19]:
from sklearn.metrics import roc_auc_score, accuracy_score
train_predictions_proba = model.predict_proba(X)[:, 1]

train_roc_auc = roc_auc_score(y, train_predictions_proba)
print(f"ROC AUC score on training data: {train_roc_auc:.4f}")

train_predictions_class = (train_predictions_proba > 0.5).astype(int)

train_accuracy = accuracy_score(y, train_predictions_class)
print(f"Accuracy score on training data: {train_accuracy:.4f}")

ROC AUC score on training data: 0.9421
Accuracy score on training data: 0.9161
